In [ ]:
# install the necessary libraries
!pip install crewai crewai_tools openai langchain_community langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Get

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Session_1

/content/drive/MyDrive/Session_1


In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')


In [ ]:
from google.colab import userdata
userdata.get('OPENAI_API_KEY')

In [ ]:
choose_LLM = 2   # 1 = OpenAI, 2 = Gemini, 3 = Ollama
colab_exp = 1    # 1 = Colab, 0 = local

import os

# --------------------------
# Load keys depending on environment
# --------------------------
if colab_exp:
    from google.colab import userdata
else:
    from dotenv import load_dotenv
    load_dotenv()


In [ ]:
# Uncomment these lines in the Gemini section to see available models:
print("Available models:")
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

Available models:
models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-use-preview-10-2025


In [ ]:
import os
# --------------------------
# Auto-detect Colab
# --------------------------
try:
    import google.colab
    COLAB = True
except ImportError:
    COLAB = False
# Load .env if local
if not COLAB:
    from dotenv import load_dotenv
    load_dotenv()
# --------------------------
# LLM selection: 1=OpenAI, 2=Gemini, 3=Ollama
# --------------------------
choose_LLM = 2  # 1=OpenAI, 2=Gemini, 3=Ollama
# --------------------------
# OpenAI
# --------------------------
if choose_LLM == 1:
    from openai import OpenAI
    if COLAB:
        from google.colab import userdata
        api_key = userdata.get("OPENAI_API_KEY")
    else:
        api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        raise ValueError("OPENAI_API_KEY is not set")
    client = OpenAI(api_key=api_key)
    llm_model = "gpt-4o-mini"
    llm = None
# --------------------------
# Gemini
# --------------------------
elif choose_LLM == 2:
    import google.generativeai as genai
    if COLAB:
        from google.colab import userdata
        api_key = userdata.get("GEMINI_API_KEY")
    else:
        api_key = os.getenv("GEMINI_API_KEY")
    if not api_key:
        raise ValueError("GEMINI_API_KEY is not set")
    genai.configure(api_key=api_key)

    # List available models (uncomment to debug)
    # print("Available models:")
    # for m in genai.list_models():
    #     if 'generateContent' in m.supported_generation_methods:
    #         print(m.name)

    # Try these model names in order of preference
    llm_model = "gemma-3-1b-it"  # Most common working model
    client = genai.GenerativeModel(llm_model)
    llm = None
# --------------------------
# Local Ollama
# --------------------------
elif choose_LLM == 3:
    from langchain.llms import Ollama
    ollama_url = "http://localhost:11434"
    llm = Ollama(model="llama3.1", base_url=ollama_url)
    client = None
    llm_model = None
else:
    raise ValueError("Invalid choose_LLM. Must be 1, 2, or 3.")
# --------------------------
# Unified query function
# --------------------------
def query_llm(prompt):
    try:
        if choose_LLM == 1:
            # OpenAI chat
            response = client.chat.completions.create(
                model=llm_model,
                messages=[{"role": "user", "content": prompt}]
            )
            return response.choices[0].message.content
        elif choose_LLM == 2:
            # Gemini - correct syntax
            response = client.generate_content(prompt)
            return response.text
        elif choose_LLM == 3:
            # Ollama
            return llm.invoke(prompt)
    except Exception as e:
        print(f"Error querying LLM: {e}")
        return None
# --------------------------
# Test
# --------------------------
if __name__ == "__main__":
    prompt = "Write a short description about CrewAI."
    output = query_llm(prompt)
    if output:
        print("LLM Output:")
        print(output)

LLM Output:
Okay, here's a short description of CrewAI, aiming for a balance of informative and engaging:

**CrewAI is a cutting-edge AI platform designed to dramatically boost team productivity and collaboration.** It leverages advanced natural language processing and machine learning to intelligently automate tasks, streamline communication, and provide real-time insights into team performance. Think of it as a smart assistant that helps teams work smarter, not harder, by intelligently managing workflows, scheduling, and knowledge sharing. **Whether you're looking to improve project management, enhance communication, or simply get more done, CrewAI offers a powerful solution.**

---

**Do you want me to:**

*   Expand on a specific aspect (e.g., its features, target audience)?
*   Tailor the description to a particular use case (e.g., for a business audience)?


In [ ]:
from IPython.display import display, Markdown, display_png

display(Markdown(output))


Okay, here's a short description of CrewAI, aiming for a balance of informative and engaging:

**CrewAI is a cutting-edge AI platform designed to dramatically boost team productivity and collaboration.** It leverages advanced natural language processing and machine learning to intelligently automate tasks, streamline communication, and provide real-time insights into team performance. Think of it as a smart assistant that helps teams work smarter, not harder, by intelligently managing workflows, scheduling, and knowledge sharing. **Whether you're looking to improve project management, enhance communication, or simply get more done, CrewAI offers a powerful solution.**

---

**Do you want me to:**

*   Expand on a specific aspect (e.g., its features, target audience)?
*   Tailor the description to a particular use case (e.g., for a business audience)?